In [1]:
!pip install pydotplus
!pip install dtreeviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 227 kB/s 


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.feature_selection import RFE


In [3]:
from sklearn import datasets, tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
import pandas as pd
import seaborn as sns


In [4]:
from google.colab import files

uploaded = files.upload()

Saving bdpredicao.xlsx to bdpredicao.xlsx


In [5]:
df = pd.read_excel(next(iter(uploaded.keys())))
df

,id_cooperado,pf_ou_pj,especialidade,competencia,nacionalidade,situacao_epoca_do_repasse,hospital,estado_civil,grau_instrucao,cooperado_origem_de_conversao,cooperado_destino_de_conversao,participa_servico_especial,valor_procedimento_a_repassar,valor_lancamento_eventual_a_repassar,valor_predicao_liquido
0,1719,pf,Médico cirurgião geral,2021-01-01,Brasileiro(a),LICENCIADO,P ATENDIMENTO,Solteiro,Superior,0,0,0,0.0,0.0,0.0
1,1719,pf,Médico cirurgião geral,2021-02-01,Brasileiro(a),LICENCIADO,P ATENDIMENTO,Solteiro,Superior,0,0,0,0.0,0.0,0.0
2,1719,pf,Médico cirurgião geral,2021-03-01,Brasileiro(a),LICENCIADO,P ATENDIMENTO,Solteiro,Superior,0,0,0,0.0,0.0,0.0
3,1719,pf,Médico cirurgião geral,2021-04-01,Brasileiro(a),LICENCIADO,P ATENDIMENTO,Solteiro,Superior,0,0,0,0.0,0.0,0.0
4,1719,pf,Médico cirurgião geral,2021-05-01,Brasileiro(a),LICENCIADO,P ATENDIMENTO,Solteiro,Superior,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74479,24883,pf,Médico generalista,2022-02-01,Brasileiro(a),ATIVO,CONSULTORIOS,Casado,Superior,0,0,0,0.0,0.0,0.0
74480,24883,pf,Médico generalista,2022-03-01,Brasileiro(a),ATIVO,CONSULTORIOS,Casado,Superior,0,0,0,0.0,0.0,0.0
74481,24883,pf,Médico generalista,2022-04-01,Brasileiro(a),ATIVO,CONSULTORIOS,Casado,Superior,0,0,0,0.0,0.0,0.0
74482,24883,pf,Médico generalista,2022-05-01,Brasileiro(a),ATIVO,CONSULTORIOS,Casado,Superior,0,0,0,0.0,0.0,0.0


In [6]:


le = preprocessing.LabelEncoder()


In [7]:
df['pf_ou_pj'].isnull().sum()


0

In [8]:
df['competencia'].isnull().sum()

0

Registros sem nacionalidade foram setados com a moda 

In [9]:
df['nacionalidade'].fillna(('Brasileira'), inplace=True)

Como se tratam de dados sem especialidades, preferi criar um item que demonstra que não tem especialidade. Esses dados possuem valores de procedimentos e lançamentos eventuais que não podem ser ignorados. Também se trata de registros com definição difícil de fazer.

In [10]:
df['especialidade'].fillna(('Sem Especialidade'), inplace=True)


In [11]:
df['situacao_epoca_do_repasse'].isnull().sum()

0

In [12]:
df['hospital'].isnull().sum()

5436

Como se tratam de dados sem hospitais, 
preferi criar um item  que demonstra que não tem hospital. Esses dados possuem valores de procedimentos e 
lançamentos eventuais que não podem ser ignorados. Também se trata de registros com definição difícil de fazer.

In [13]:
df['hospital'].fillna(('Sem Hospital'), inplace=True)

Como se trata de dados sem estado civíl, 
preferi criar um estado civíl com descrição que demonstra que não tem informação por possuir alguns valores de procedimentos e 
lançamentos eventuais que não podem ser ignorados. Podemos descobrir algum conhecimento aqui no que se refere a ter mais tempo para atuar

In [14]:
df['estado_civil'].fillna(('Sem'), inplace=True)

Registros sem grau de instrução se referem a cadastros de empresas.
Será adotado um ítem pj nesse caso.

In [15]:

df['grau_instrucao'].fillna(('pj'), inplace=True)

Atributos categóricos que foram convertidos utilizando one hot encoding. 
Como não se tratam de features que tem sentido de ordenação,foi escolhido
o one hot encoding para que o modelo não interprete uma relação de ordem.
Podemos ter problemas com memória nesse caso mas inicialmente trabalharemos da forma ensinada 
na literatura.

In [16]:
def generate_labelencoder(atts):
  for attr in atts:
    df[attr] = le.fit_transform(df[attr])
  return df

In [17]:
df['vai_receber'] = np.where(df['competencia']=='2022-06-01',2, 
         (np.where(df['valor_predicao_liquido'] > 0.0000000001, 1, 0)))

Features categóricas com domínio binário ou com sentido de ordenação. Convertidas utilizando o Label Encoder

In [18]:

df  = generate_labelencoder(['pf_ou_pj','grau_instrucao','competencia','hospital','especialidade','estado_civil','nacionalidade','situacao_epoca_do_repasse'])

In [19]:
df = df.drop(['id_cooperado'], axis=1)

In [20]:
df_classificador = df

In [21]:
df_classificador['valor_total'] = df_classificador['valor_procedimento_a_repassar']+df_classificador['valor_lancamento_eventual_a_repassar']

In [22]:
df_classificador = df_classificador.drop(['valor_lancamento_eventual_a_repassar'], axis=1)
df_classificador = df_classificador.drop(['pf_ou_pj'], axis=1)
df_classificador = df_classificador.drop(['valor_procedimento_a_repassar'], axis=1)
df_classificador = df_classificador.drop(['especialidade'], axis=1)
df_classificador = df_classificador.drop(['nacionalidade'], axis=1)
df_classificador = df_classificador.drop(['hospital'], axis=1)
df_classificador = df_classificador.drop(['estado_civil'], axis=1)
df_classificador = df_classificador.drop(['grau_instrucao'], axis=1)
df_classificador = df_classificador.drop(['cooperado_origem_de_conversao'], axis=1)
df_classificador = df_classificador.drop(['cooperado_destino_de_conversao'], axis=1)
df_classificador = df_classificador.drop(['participa_servico_especial'], axis=1)

In [23]:
df_competencia_junho_2022 = df_classificador[df_classificador["vai_receber"]==2]

In [24]:
df_classificador = df_classificador.drop(df_classificador.loc[df_classificador['vai_receber']==2].index)

In [25]:
X = df_classificador.loc[:, df_classificador.columns != 'valor_predicao_liquido']
X = X.loc[:, X.columns != 'vai_receber']
X = X.query("competencia!='2022-06-01'")
Y = df_classificador.query("competencia!='2022-06-01'")
Y = Y.loc[:, Y.columns == 'vai_receber']


In [26]:
X_predict = df_competencia_junho_2022.loc[:, df_classificador.columns != 'valor_predicao_liquido']
X_predict = X_predict.loc[:, X_predict.columns != 'vai_receber']



In [27]:
################################################
X_Resto = X.loc[:, X.columns != 'valor_total']
X_Total = X.loc[:, X.columns == 'valor_total']

np.seterr(divide = 'ignore')



# Normalização Min-Max dos dados.

min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
X_Total['valor_total'] = min_max_scaler.fit_transform(X_Total)
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
X_Resto[['competencia','situacao_epoca_do_repasse']] = min_max_scaler.fit_transform(X_Resto)




#Y = min_max_scaler.fit_transform(Y)

<ipython-input-27-4594ffbe4be3>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_Total['valor_total'] = min_max_scaler.fit_transform(X_Total)
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [28]:
################################################
X_Resto_predict = X_predict.loc[:, X_predict.columns != 'valor_total']
X_Total_predict = X_predict.loc[:, X_predict.columns == 'valor_total']

np.seterr(divide = 'ignore')



# Normalização Min-Max dos dados.

min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
X_Total_predict['valor_total'] = min_max_scaler.fit_transform(X_Total_predict)
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
X_Resto_predict[['competencia','situacao_epoca_do_repasse']] = min_max_scaler.fit_transform(X_Resto_predict)




#Y = min_max_scaler.fit_transform(Y)

<ipython-input-28-03df7e69a015>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_Total_predict['valor_total'] = min_max_scaler.fit_transform(X_Total_predict)
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [29]:
#X = np.concatenate((resto, valor_total), axis=1)

In [30]:
X = X_Resto.join(X_Total)

In [31]:
X_predict = X_Resto_predict.join(X_Total_predict)

In [32]:
x_train, x_test, y_train, y_test = train_test_split( X, Y.vai_receber, test_size=0.2)

In [33]:
gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train).predict(x_test)

In [34]:
print("Número de erros de classificação do modelo Naive Bayes {0} de {1}"
      .format((y_test != y_pred).sum(), y_pred.shape))

ac = gnb.score(x_test, y_test)

print("\nAcurácia do modelo: {0:.2f}%\n".format(100*ac))

Número de erros de classificação do modelo Naive Bayes 398 de (14070,)

Acurácia do modelo: 97.17%



In [35]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm = svm.fit(x_train,y_train)
yprevisao = svm.predict(x_test)

In [36]:
from sklearn import metrics
metrics.accuracy_score(y_test, yprevisao)

0.8056147832267235

In [37]:
print("Número de erros de classificação do modelo SVM {0} de {1}"
      .format((y_test != yprevisao).sum(), yprevisao.shape))

Número de erros de classificação do modelo SVM 2735 de (14070,)


In [38]:

tree_recebe_nao_recebe = DecisionTreeClassifier(random_state=0, criterion='entropy',min_samples_split =100,max_depth=10) 
tree_recebe_nao_recebe = tree_recebe_nao_recebe.fit(x_train, y_train)
print("Acurácia (base de treinamento):", tree_recebe_nao_recebe.score(x_train, y_train))

Acurácia (base de treinamento): 0.9875968441253821


In [39]:
print("Número de erros de classificação {0} de {1}"
      .format((y_test != y_pred).sum(), y_pred.shape))

Número de erros de classificação 398 de (14070,)


In [40]:
y_pred = tree_recebe_nao_recebe.predict(x_test)
print("Acurácia de previsão:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['NAO','SIM']))


Acurácia de previsão: 0.9869225302061123
              precision    recall  f1-score   support

         NAO       0.98      1.00      0.99     10795
         SIM       1.00      0.95      0.97      3275

    accuracy                           0.99     14070
   macro avg       0.99      0.97      0.98     14070
weighted avg       0.99      0.99      0.99     14070



In [41]:
y_predict = tree_recebe_nao_recebe.predict(X_predict)


In [42]:
X_predict['vai_receber'] =y_predict

In [43]:
x_train_test = X.join(Y)

In [44]:
df_apos_classificacao=pd.concat([X_predict,x_train_test])

In [45]:
df['vai_receber']=df_apos_classificacao['vai_receber']

In [46]:
import itertools
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time

params = {'min_samples_split': [20,30,40,50,100],
          'max_depth': [10,15,20,30,40,50],
          'criterion': ['gini','entropy']
         }
decision_tree = GridSearchCV(DecisionTreeClassifier(random_state=0), param_grid=params, n_jobs=-1, cv=5, verbose=5)
decision_tree.fit(x_train,y_train)

print('Train R^2 Score : %.3f'%decision_tree.best_estimator_.score(x_train, y_train))
print('Test R^2 Score : %.3f'%decision_tree.best_estimator_.score(x_test, y_test))
print('Best R^2 Score Through Grid Search : %.3f'%decision_tree.best_score_)
print('Best Parameters : ',decision_tree.best_params_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Train R^2 Score : 0.987
Test R^2 Score : 0.988
Best R^2 Score Through Grid Search : 0.988
Best Parameters :  {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 20}
CPU times: user 1.13 s, sys: 187 ms, total: 1.31 s
Wall time: 9.63 s


In [47]:
df_regressao = df

In [48]:
del df_regressao['valor_lancamento_eventual_a_repassar']
del df_regressao['valor_procedimento_a_repassar']

In [49]:
X_feature_selection = df_regressao.query("competencia!=17 and vai_receber==1")
X_feature_selection = X_feature_selection.loc[:, X_feature_selection.columns != 'valor_predicao_liquido']

Y_feature_selection = df_regressao.query("competencia!=17 and vai_receber==1")
Y_feature_selection = Y_feature_selection.loc[:, Y_feature_selection.columns == 'valor_predicao_liquido']

In [50]:
del X_feature_selection['vai_receber']

In [51]:
x_train, x_test, y_train, y_test = train_test_split( X_feature_selection, Y_feature_selection, test_size=0.2)

In [52]:
X_regressao = df_regressao.query("competencia!=17 and vai_receber==1")
X_regressao = X_regressao.loc[:, X_regressao.columns != 'valor_predicao_liquido']

Y_regressao = df_regressao.query("competencia!=17 and vai_receber==1")
Y_regressao = Y_regressao.loc[:, Y_regressao.columns == 'valor_predicao_liquido']

In [53]:
X_regressao

,pf_ou_pj,especialidade,competencia,nacionalidade,situacao_epoca_do_repasse,hospital,estado_civil,grau_instrucao,cooperado_origem_de_conversao,cooperado_destino_de_conversao,participa_servico_especial,vai_receber,valor_total
19,0,52,1,3,0,11,0,2,1,0,0,1,172.88
20,0,52,2,3,0,11,0,2,1,0,0,1,310.10
21,0,52,3,3,0,11,0,2,1,0,0,1,347.65
22,0,52,4,3,0,11,0,2,1,0,0,1,300.65
28,0,52,10,3,0,11,0,2,1,0,0,1,18.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73906,1,27,16,2,0,17,4,3,0,1,0,1,4872.95
73942,0,58,16,3,0,7,6,2,0,0,0,1,822.80
73960,1,40,16,2,0,17,4,3,0,1,0,1,10714.33
74104,0,43,16,3,0,6,6,2,0,0,0,1,2500.00


In [54]:
X_regressao = X_regressao[['pf_ou_pj', 'especialidade', 'competencia', 'hospital',
       'grau_instrucao', 'valor_total']]

In [55]:
X_predicao_da_regressao = df_regressao.query("competencia==17 and vai_receber==1")
X_predicao_da_regressao = X_predicao_da_regressao.loc[:, X_predicao_da_regressao.columns != 'valor_predicao_liquido']


In [56]:
X_predicao_da_regressao = X_predicao_da_regressao[['pf_ou_pj', 'especialidade', 'competencia', 'hospital',
       'grau_instrucao', 'valor_total']]

In [57]:

X_Resto = X_regressao.loc[:, X_regressao.columns != 'valor_total']
X_Total = X_regressao.loc[:, X_regressao.columns == 'valor_total']

np.seterr(divide = 'ignore')



# Normalização Min-Max dos dados.


min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
valor_total = min_max_scaler.fit_transform(X_Total)
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
resto = min_max_scaler.fit_transform(X_Resto)




#Y = min_max_scaler.fit_transform(Y)

In [58]:

X_Resto_predicao_regressao = X_predicao_da_regressao.loc[:, X_predicao_da_regressao.columns != 'valor_total']
X_Total_predicao_regressao = X_predicao_da_regressao.loc[:, X_predicao_da_regressao.columns == 'valor_total']

np.seterr(divide = 'ignore')


min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
valor_total_predicao_regressao = min_max_scaler.fit_transform(X_Total_predicao_regressao)
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
resto_predicao_regressao = min_max_scaler.fit_transform(X_Resto_predicao_regressao)



In [59]:
X = np.concatenate((resto, valor_total), axis=1)

In [60]:
X_predicao_regressao = np.concatenate((resto_predicao_regressao, valor_total_predicao_regressao), axis=1)

In [215]:
x_train, x_test, y_train, y_test = train_test_split( X, Y_regressao, test_size=0.2)

In [216]:
mlp = MLPRegressor(random_state=123,activation='relu', alpha=0.001, max_iter=5000,hidden_layer_sizes= (20),epsilon=1e-08,  verbose=True,solver='lbfgs',learning_rate='constant')
mlp.fit(x_train, y_train)

MLPRegressor(alpha=0.001, hidden_layer_sizes=20, max_iter=5000,
             random_state=123, solver='lbfgs', verbose=True)

In [217]:
pred = mlp.predict(x_test)

In [218]:
from sklearn.metrics import r2_score


In [219]:
print('acuracia na base de Teste ' + str(metrics.r2_score(y_test.to_numpy(), pred)))

acuracia na base de Teste 0.9961175247267366


In [220]:
from sklearn.metrics import mean_squared_error, r2_score #Métricas para avaliar a regressÃ£o

In [221]:
print("Erro quadrÃ¡tico MÃ©dio = ",mean_squared_error(y_test, pred)) #Erro quadrático médio
print("R2 =", r2_score(y_test, pred)) #Coeficiente de determinação

Erro quadrÃ¡tico MÃ©dio =  3056147.318657154
R2 = 0.9961175247267366


In [222]:
y_test['valor_predito']=pred

In [223]:
#Com margem de erro de 10%
len(y_test[(10*abs(y_test['valor_predicao_liquido']/100))>=abs(y_test['valor_predicao_liquido']-y_test['valor_predito'])])

1221

In [224]:
#Com margem de erro de 20%
len(y_test[(20*abs(y_test['valor_predicao_liquido']/100))>=abs(y_test['valor_predicao_liquido']-y_test['valor_predito'])])

2002

In [164]:
%%time

params = {'activation': ['relu', 'tanh', 'logistic', 'identity'],
          'solver': ['adam','sgd', 'lbfgs'],
          'hidden_layer_sizes': [(1,),(10,),(20,),(30,)] ,
          'learning_rate' : ['constant', 'adaptive', 'invscaling']
         }

mlp_regressor_grid = GridSearchCV(MLPRegressor(random_state=123, alpha=0.001, ), param_grid=params, n_jobs=-1, cv=5, verbose=5)
mlp_regressor_grid.fit(x_train,y_train)

print('Train R^2 Score : %.3f'%mlp_regressor_grid.best_estimator_.score(x_train, y_train))
print('Test R^2 Score : %.3f'%mlp_regressor_grid.best_estimator_.score(x_test, y_test))
print('Best R^2 Score Through Grid Search : %.3f'%mlp_regressor_grid.best_score_)
print('Best Parameters : ',mlp_regressor_grid.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Train R^2 Score : 0.998
Test R^2 Score : 0.998
Best R^2 Score Through Grid Search : 0.997
Best Parameters :  {'activation': 'relu', 'hidden_layer_sizes': (20,), 'learning_rate': 'constant', 'solver': 'lbfgs'}
CPU times: user 16.2 s, sys: 4.08 s, total: 20.3 s
Wall time: 23min 17s


In [207]:
x_train, x_test, y_train, y_test = train_test_split( X, Y_regressao, test_size=0.2)

In [186]:
%%time
from sklearn.linear_model import Ridge


params = {'alpha': [0.01,0.1,1,10]
         }
ridgeReg = GridSearchCV(Ridge(random_state=0), param_grid=params, n_jobs=-1, cv=5, verbose=5)
ridgeReg.fit(x_train,y_train)

print('Train R^2 Score : %.3f'%ridgeReg.best_estimator_.score(x_train, y_train))
print('Test R^2 Score : %.3f'%ridgeReg.best_estimator_.score(x_test, y_test))
print('Best R^2 Score Through Grid Search : %.3f'%ridgeReg.best_score_)
print('Best Parameters : ',ridgeReg.best_params_)


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Train R^2 Score : 0.997
Test R^2 Score : 0.991
Best R^2 Score Through Grid Search : 0.997
Best Parameters :  {'alpha': 0.1}
CPU times: user 87.4 ms, sys: 18.6 ms, total: 106 ms
Wall time: 192 ms


In [208]:
from sklearn.linear_model import Ridge

ridgeReg = Ridge(alpha=0.1).fit(x_train,y_train)


In [209]:
pred = ridgeReg.predict(x_test)

mse = np.mean((pred - y_test)**2)

mse

valor_predicao_liquido    954655.311135
dtype: float64

In [210]:
print('acuracia na base de treinamento ' + str(ridgeReg.score(x_train,y_train)))
print('acuracia na base de teste '+ str(ridgeReg.score(x_test,y_test)))

acuracia na base de treinamento 0.9962327821047537
acuracia na base de teste 0.9978246990585917


In [211]:
y_test['valor_predito']=pred

In [213]:
#Ridge com com margem de erro de 10%
len(y_test[(10*abs(y_test['valor_predicao_liquido']/100))>=abs(y_test['valor_predicao_liquido']-y_test['valor_predito'])])

670

In [214]:
#Ridge com margem de erro de 20%
len(y_test[(20*abs(y_test['valor_predicao_liquido']/100))>=abs(y_test['valor_predicao_liquido']-y_test['valor_predito'])])

1117

In [247]:
x_train, x_test, y_train, y_test = train_test_split(X, Y_regressao, test_size=0.2)

In [250]:
from sklearn.linear_model import Lasso

lasso = Lasso().fit(x_train, y_train)
pred = lasso.predict(x_test)
print("Lasso alpha=1")
print("Erro quadrÃ¡tico MÃ©dio = ",mean_squared_error(y_test, pred)) #Erro quadrático médio
print("R2 =", r2_score(y_test, pred)) #Coeficiente de determinação
print("Número de atributos usados: {}".format(np.sum(lasso.coef_ != 0)))


lasso001 = Lasso(alpha=0.01, max_iter=100000).fit(x_train, y_train)
pred = lasso001.predict(x_test)
print("Lasso alpha=.01")

print("Erro quadrÃ¡tico MÃ©dio = ",mean_squared_error(y_test, pred)) #Erro quadrático médio
print("R2 =", r2_score(y_test, pred)) #Coeficiente de determinação
print("Número de atributos usados: {}".format(np.sum(lasso001.coef_ != 0)))


lasso00001 = Lasso(alpha=0.0001, max_iter=100000).fit(x_train, y_train)
pred = lasso00001.predict(x_test)
print("Lasso alpha=.0001")
print("Erro quadrÃ¡tico MÃ©dio = ",mean_squared_error(y_test, pred)) #Erro quadrático médio
print("R2 =", r2_score(y_test, pred)) #Coeficiente de determinação
print("Número de atributos usados: {}".format(np.sum(lasso00001.coef_ != 0)))


Lasso alpha=1
Erro quadrÃ¡tico MÃ©dio =  1149160.5777263104
R2 = 0.9967071527441639
Número de atributos usados: 6
Lasso alpha=.01
Erro quadrÃ¡tico MÃ©dio =  1158415.4117383966
R2 = 0.9966806336002161
Número de atributos usados: 6
Lasso alpha=.0001
Erro quadrÃ¡tico MÃ©dio =  1158512.1438790841
R2 = 0.9966803564203596
Número de atributos usados: 6


In [251]:
lasso = Lasso().fit(x_train, y_train)
pred = lasso.predict(x_test)
print("Lasso alpha=1")
print("Erro quadrÃ¡tico MÃ©dio = ",mean_squared_error(y_test, pred)) #Erro quadrático médio
print("R2 =", r2_score(y_test, pred)) #Coeficiente de determinação
print("Número de atributos usados: {}".format(np.sum(lasso.coef_ != 0)))

Lasso alpha=1
Erro quadrÃ¡tico MÃ©dio =  1149160.5777263104
R2 = 0.9967071527441639
Número de atributos usados: 6


In [252]:
y_test['valor_predito']=pred

In [253]:
#Com margem de erro de 10%
len(y_test[(10*abs(y_test['valor_predicao_liquido']/100))>=abs(y_test['valor_predicao_liquido']-y_test['valor_predito'])])

724